In [7]:
import json
file_path = "data/qwen_cnn_test_data.json"
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [8]:
all_result = []
all_result.extend(data[:50])

In [9]:
len(all_result)

50

In [10]:
file_path = "data/qwen_cnn_50.json"
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(all_result, f, ensure_ascii=False, indent=4)

In [2]:
from safetensors.torch import load_file  # safetensors 파일 로드
import torch

# safetensors 파일을 사용할 경우
model_path = "model/mean/checkpoint-8000/adapter_model.safetensors"
model_weights = load_file(model_path)

# 만약 PyTorch .bin 파일로 저장된 경우
# model_path = "checkpoint-8000/adapter_model.bin"
# model_weights = torch.load(model_path)

# 저장된 가중치의 키(파라미터 이름) 확인
for param_name in model_weights.keys():
    print(param_name)

# 특정 파라미터가 저장되었는지 확인하고자 할 때
# 관련 파라미터 확인
if "test" in model_weights:
    print("파라미터가 저장되었습니다.")
else:
    print("파라미터가 저장되지 않았습니다.")

base_model.model.lm_head.weight
base_model.model.model.embed_tokens.weight
base_model.model.model.layers.0.self_attn.k_proj.lora_A.weight
base_model.model.model.layers.0.self_attn.k_proj.lora_B.weight
base_model.model.model.layers.0.self_attn.o_proj.lora_A.weight
base_model.model.model.layers.0.self_attn.o_proj.lora_B.weight
base_model.model.model.layers.0.self_attn.q_proj.lora_A.weight
base_model.model.model.layers.0.self_attn.q_proj.lora_B.weight
base_model.model.model.layers.0.self_attn.v_proj.lora_A.weight
base_model.model.model.layers.0.self_attn.v_proj.lora_B.weight
base_model.model.model.layers.1.self_attn.k_proj.lora_A.weight
base_model.model.model.layers.1.self_attn.k_proj.lora_B.weight
base_model.model.model.layers.1.self_attn.o_proj.lora_A.weight
base_model.model.model.layers.1.self_attn.o_proj.lora_B.weight
base_model.model.model.layers.1.self_attn.q_proj.lora_A.weight
base_model.model.model.layers.1.self_attn.q_proj.lora_B.weight
base_model.model.model.layers.1.self_attn.v